In [7]:
# +
from dataset.semi import SemiDataset
from model.semseg.deeplabv2 import DeepLabV2
from model.semseg.deeplabv3plus import DeepLabV3Plus
from model.semseg.pspnet import PSPNet
from utils import count_params, meanIOU, color_map, Accuracy, DiceCoefficient

from utilsf.loss_file import save_loss
# -

import argparse
from copy import deepcopy
import numpy as np
import os
from PIL import Image
import torch
from torch.nn import CrossEntropyLoss, DataParallel, functional as F
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import DataLoader
from tqdm import tqdm

In [15]:
semi_setting = 'kidney/0114'
save_path = f'outdir/models/{semi_setting}'
pseudo_mask_path = f'outdir/pseudo_masks/{semi_setting}'
labeled_id_path = f'dataset/splits/{semi_setting}/labeled.txt'
reliable_id_path = f'outdir/reliable_ids/{semi_setting}'
data_root = '/tf/dataset/0_data_dataset_voc_950_kidney'
batch_size = 4
crop_size = 224
plus = True
dataset ='kidney'

if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(pseudo_mask_path):
    os.makedirs(pseudo_mask_path)
if plus and reliable_id_path is None:
    exit('Please specify reliable-id-path in ST++.')

criterion = CrossEntropyLoss(ignore_index=255)

valset = SemiDataset(dataset, data_root, 'val', None)
valloader = DataLoader(valset, batch_size=4 if dataset == 'cityscapes' else 1,
                       shuffle=False, pin_memory=True, num_workers=4, drop_last=False)

In [16]:
MODE = 'train'

trainset = SemiDataset(dataset, data_root, MODE, crop_size, labeled_id_path)
trainset.ids = 2 * trainset.ids if len(trainset.ids) < 200 else trainset.ids
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True,
                         pin_memory=True, num_workers=16, drop_last=True)

In [17]:
def init_basic_elems():
    model_zoo = {'deeplabv3plus': DeepLabV3Plus, 'pspnet': PSPNet, 'deeplabv2': DeepLabV2}
    model = model_zoo['deeplabv3plus']('resnet18', 2 if dataset == 'kidney' else 19)

    head_lr_multiple = 10.0

    optimizer = SGD([{'params': model.backbone.parameters(), 'lr': 0.001},
                     {'params': [param for name, param in model.named_parameters()
                                 if 'backbone' not in name],
                      'lr': 0.001 * head_lr_multiple}],
                    lr=0.001, momentum=0.9, weight_decay=1e-4)

    model = DataParallel(model).cuda()

    return model, optimizer

In [18]:
model, optimizer = init_basic_elems()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, pred, targets, smooth=1):
        target_one_hot = F.one_hot(targets, num_classes=pred.shape[1]).permute(0, 3, 1, 2).float()
        target_one_hot = target_one_hot[:, 1:, :, :]
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        pred = pred[:, 1:, :, :]  # shape: (16, C-1, 224, 224), e.g. (16, 3, 224, 224) -> (16, 2, 224, 224)
        pred = F.sigmoid(pred)
        
        #flatten label and prediction tensors
        pred_flat = pred.contiguous().view(-1)
        target_flat = target_one_hot.contiguous().view(-1)
        
        intersection = (pred_flat * target_flat).sum()
        union = pred_flat.sum() + target_flat.sum()

        dice = (2. * intersection + smooth) / (union + smooth)
        return 1 - dice

In [20]:
criterion = DiceLoss()

In [21]:
for img, mask in trainloader:
    img, mask = img.to(device), mask.to(device)
    pred = model(img)  
    break

In [22]:
print('img shape: ', img.shape)
print('mask shape: ', mask.shape)
print('pred shape: ', pred.shape)

# 檢查 masks 的最小值和最大值
print('mask min value: ', mask.min().item())
print('mask max value: ', mask.max().item())

print('pred min value: ', pred.min().item())
print('pred max value: ', pred.max().item())

img shape:  torch.Size([4, 3, 224, 224])
mask shape:  torch.Size([4, 224, 224])
pred shape:  torch.Size([4, 2, 224, 224])
mask min value:  0
mask max value:  0
pred min value:  -6.024209499359131
pred max value:  1.7685109376907349


In [24]:
loss = criterion(pred, mask)

print('Loss: ', loss.item())

Loss:  0.9999886751174927


In [ ]:
import matplotlib.pyplot as plt
from torch.nn import CrossEntropyLoss, DataParallel, functional as F


# 可視化 pred_classes 的第一個樣本
pred_class_sample = pred[0].detach().cpu().numpy()

plt.figure(figsize=(10, 5))
plt.imshow(pred_class_sample, cmap='viridis')
plt.colorbar()
plt.title('Predicted Classes')
plt.show()